# Store Traffic Monitor
The object flaw detector is a reference implementation for video inferencing where the task is to track human activity an count inventory in a facility. In this example, we will see how we can scale detection at the edge to multiple devices.

This demo assumes you have already tried the object detector sample. If you have not done so already, we recommend going through that sample first. 

More information on using the traffic monitor, as well as the original source code can be found on github:

https://github.com/intel-iot-devkit/store-traffic-monitor-python

Note that the implementation here has been slightly modified to work on IoT DevCloud.

## Step 0: Setup

Import dependencies for the notebook by running the following cell

In [ ]:
from demoutils import videoHTML, liveQstat

## Step 1: Advanced OpenVINO

I was not able to get to this. Can you add the OpenVINO features and concepts that make this advanced compared to the object detector.

## Step 2: Running on the IoT DevCloud

### 2.1 Creating job file

First, we need to create a job file.
The job file is written in bash, and will be executed on the compute node.
For this example, we have written the job file for you in the notebook.
Run the following cell to write this in to the file "store_traffic_manager_job.sh"

In [ ]:
%%writefile store_traffic_monitor_job.sh

# The default path for the job is your home directory, so we change directory to where the files are.
cd $PBS_O_WORKDIR

# Traffic monitor script writes output to a file inside a directory. We make sure that this directory exists.
#  The output directory is the first argument of the bash script
mkdir -p $1

# Traffic monitor takes 3 inputs, which are passed in as arguments to the bash script. 
#  -o : output directory of the videos
#  -d : device to use for inference
#  -c : conf file to use
python3 store_traffic_monitor.py -o $1 -d $2 -c $3

### 2.2 Submitting to Job Queue

Now that we have the job script, we can submit the jobs to compute nodes using the `qsub` command.
Let's try submitting the store_traffic_monitor job to 4 different types of nodes.

There are three options of `qsub` command that we use for this:
- `-l` : this option lets us select the number and the type of nodes using `nodes={node_count}:{property}`. 
- `-F` : this option lets us send arguments to the bash script. 
- `-N` : this option lets us name the job so that it is easier to distinguish between them.

To see the available types of nodes (e.g. properties) on the IoT DevCloud, run the following cell.

In [ ]:
!pbsnodes | grep properties | sort | uniq -c

Here, the properties describe the node, and number on the left is the number of available nodes.

Now let's actually submit the jobs. 
Each of the next 4 cells to will submit jobs to different nodes.
The output of the cell is the `JobID` of your job, which you can use to track progress of a job.

#### submitting to a node with Intel Core CPU

In [ ]:
!qsub store_traffic_monitor_job.sh -l nodes=1:iei-tank-core -F "results/core CPU conf_fp32.txt" -N monitor_core

#### submitting to a node with Intel Xeon CPU

In [ ]:
!qsub store_traffic_monitor_job.sh -l nodes=1:iei-tank-xeon -F "results/xeon CPU conf_fp32.txt" -N monitor_xeon

#### submitting to a node with Intel Core CPU and using the onboard Intel GPU

In [ ]:
!qsub store_traffic_monitor_job.sh -l nodes=1:iei-tank-core -F "results/gpu GPU conf_fp32.txt" -N monitor_gpu

#### submitting to a node with Intel Movidius Stick

In [ ]:
!qsub store_traffic_monitor_job.sh -l nodes=1:iei-tank-movidius -F "results/myriad MYRIAD conf_fp16.txt" -N monitor_myriad

### 2.3 Check if the jobs are done

Run the following cell to bring the custom qstat widget. 

In [ ]:
liveQstat()

You should see the jobs you have submitted (referenced by `Job ID`).
It should also show the jupyter notebook job as well. 

## Step 3: Results

Obce the jobs are complete, the stdout and stderr are store in files with names of the form (based on our `-N` argument):

`monitor_{type}.o{JobID}`

`monitor_{type}.e{JobID}`

But for this script, the main output is the mp4 videos which are stored in the `results/` directory.
We wrote a short utility script that will display these videos in the notebook.
See `demoutils.py` if interested in the script.

Then run the following four cells to see the output  from out jobs.

In [ ]:
videoHTML('IEI Tank (Intel Core CPU)', 'results/core/', 'results/core/fps.txt')

In [ ]:
videoHTML('IEI Tank Xeon (Intel Xeon CPU)','results/xeon/', 'results/xeon/fps.txt')

In [ ]:
videoHTML('IEI Inel GPU (Intel Onboard GPU)', 'results/gpu/', 'results/gpu/fps.txt')

In [ ]:
videoHTML('IEI Tank + Myriad (Intel Core + Movidius)','results/myriad/', 'results/myriad/fps.txt')